In [1]:
import pandas as pd
import numpy as pn
import sklearn as sk
import json
import matplotlib.pyplot as plt

In [2]:
# movies = pd.read_csv('./data/head_10_movies.csv')
# credits = pd.read_csv('./data/head_10_credits.csv')
movies = pd.read_csv('./data/tmdb_5000_movies.csv').head(n=1)
credits = pd.read_csv('./data/tmdb_5000_credits.csv').head(n=1)

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
movies = pd.merge(left=movies, right=credits, left_on='id', right_on='movie_id')

In [6]:
# turn json into python objects
json_columns = ['genres', 'keywords', 'production_countries', 'spoken_languages', 'cast', 'crew']
for column in json_columns:
    movies[column] = movies[column].apply(json.loads, encoding="utf-8")

In [7]:
movies.genres[0][0]['name']

'Action'

In [8]:
def cast_to_actors(cast):
    actors = []
    for person in cast:
        actors.append('{} - {}'.format(person['name'], person['id']))
    return actors

In [9]:
movies.actors = movies.cast.apply(cast_to_actors)

In [10]:
movies.actors

0    [Sam Worthington - 65731, Zoe Saldana - 8691, ...
Name: cast, dtype: object

### Create list of all actors

In [11]:
all_actors = []
for actors in movies.actors:
    for actor in actors:
        all_actors.append(actor)

In [12]:
actors = pd.Series(all_actors)

In [13]:
unique_actors_count = pd.crosstab(actors, columns='count') \
.sort_values(by='count', ascending=False)

unique_actors_count.head(12)

col_0,count
row_0,
Alicia Vela-Bailey - 1186027,1
Peter Mensah - 68278,1
Paul Yates - 1207265,1
Nikie Zambo - 103259,1
Nicole Dionne - 147898,1
Nathan Meister - 33305,1
Mike Bodnar - 107969,1
Michelle Rodriguez - 17647,1
Michael Blain-Rozgay - 143206,1


### Vectorize actors

In [14]:
movies_actors_ids = []
for actors in movies.cast:
    movie_actors_ids = []
    for actor in actors:
        movie_actors_ids.append(str(actor['id']))
    movies_actors_ids.append(' '.join(movie_actors_ids))

    

# THIS IS BROKEN

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
CountVectorizer(analyzer='word', binary=True)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
#movies['vector_actors'] = vectorizer.fit_transform(movies_actors_ids)

In [17]:
movies['vector_actors'] = vectorizer.fit_transform(movies_actors_ids).toarray().tolist()

In [18]:
len(unique_actors_count)

83

# Instead, use dummies

In [44]:
pd.get_dummies(pd.Series(movies.actors), prefix='actors')

TypeError: unhashable type: 'list'

### label records as having Samuel L. Jackson

In [19]:
samuel_label = 'Samuel L. Jackson - 2231'
def record_has_samuel (actors):
    return samuel_label in actors


In [20]:
movies.samuel = movies.actors.apply(record_has_samuel)

In [21]:
movies_with_samuel = movies[movies.samuel == True]

#### There should be 67 movies with Samuel L Jackson

In [22]:
len(movies_with_samuel)

0

# Create Parameters

In [23]:
y = movies.samuel

In [39]:
#x = movies[['title_x', 'revenue']]
x = movies[['revenue', 'vote_average', 'vote_count', 'vector_actors']]
#x = movies[['revenue', 'vote_average', 'vote_count']]

#### We can create an x variable with what we want directly, but can not assign it to the dataframe

In [40]:
#x = vectorizer.fit_transform(movies_actors_ids).toarray()

# Predict Using Bayes

In [41]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x,y)

ValueError: setting an array element with a sequence.

## Get Accuracy

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [28]:
predict_true = clf.predict(x)

/anaconda/lib/python3.6/site-packages/sklearn/naive_bayes.py:428: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/anaconda/lib/python3.6/site-packages/sklearn/naive_bayes.py:430: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)


In [29]:
len(predict_true[predict_true == True])

0

In [30]:
confusion_matrix(y, clf.predict(x))

/anaconda/lib/python3.6/site-packages/sklearn/naive_bayes.py:428: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/anaconda/lib/python3.6/site-packages/sklearn/naive_bayes.py:430: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)


array([[1]])

In [31]:
accuracy_score(y, predict_true)

1.0

# TODO

Perform deap learning prediction https://www.kaggle.com/liwste/simple-deep-mlp-with-keras

Create cross validation

In [32]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew,vector_actors
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
## Go through the entire 